In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=c1a85693-dec2-4a47-ad23-8df81853d269
To: /kaggle/working/data.pkl
100%|██████████████████████████████████████| 1.24G/1.24G [00:46<00:00, 26.9MB/s]


In [2]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import data_filtering
from preprocess import resampling
from preprocess import RP_3D
from preprocess import resize_volume
from preprocess import RP_final_data
from preprocess import label_creator
from preprocess import including_data
from metrics import result

2024-07-06 08:17:56.513858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-06 08:17:56.513971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-06 08:17:56.638826: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [4]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 8923.74it/s]


In [5]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)
x_train, y_train, x_test, y_test = including_data(labels, ds)

100%|██████████| 10344/10344 [00:00<00:00, 930468.40it/s]
10344it [00:08, 1289.24it/s]
10344it [00:04, 2198.39it/s]


In [6]:
data_2D = []
for i in tqdm(x_train):
    tmp_2D = RP_3D(i)
    data_2D.append(np.reshape(tmp_2D, (99,99,12)))
x_train_2d = np.array(data_2D)

data_2D = []
for i in tqdm(x_test):
    tmp_2D = RP_3D(i)
    data_2D.append(np.reshape(tmp_2D, (99,99,12)))
x_test_2d = np.array(data_2D)

100%|██████████| 1126/1126 [00:06<00:00, 175.06it/s]


In [7]:
print(np.shape(x_train_2d))
print(np.shape(x_test_2d))

(5000, 99, 99, 12)
(1126, 99, 99, 12)


In [8]:
# Model

import tensorflow as tf


def madan_model():
    input_layer = tf.keras.layers.Input(shape=(99,99,12))
    x = tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='same')(input_layer)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=(1,1), padding='same')(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(256, kernel_size=(1,1), padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = tf.keras.layers.Reshape((12*12, 256))(x)
    x = tf.keras.layers.LSTM(4096)(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1000)(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(7, activation='sigmoid')(x)
    return tf.keras.models.Model(input_layer, x)

In [9]:
# Training

## Compiling
def CompModel(model):
    model.compile(optimizer=tf.keras.optimizers.Adam() , 
                loss=tf.keras.losses.BinaryCrossentropy() , metrics=['acc'])
    lr_sch = tf.keras.callbacks.LearningRateScheduler(
        lambda epochs: 1e-3 * 10 ** (-epochs/100.0))
    return model, lr_sch

model = madan_model()
model, lr_sch = CompModel(model)

history = model.fit(x_train_2d, y_train.values,
            epochs=100,
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 75s 431ms/step - acc: 0.2661 - loss: 0.4713 - learning_rate: 0.0010
Epoch 2/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 433ms/step - acc: 0.4652 - loss: 0.3520 - learning_rate: 9.7724e-04
Epoch 3/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 433ms/step - acc: 0.5249 - loss: 0.3254 - learning_rate: 9.5499e-04
Epoch 4/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 434ms/step - acc: 0.5668 - loss: 0.3047 - learning_rate: 9.3325e-04
Epoch 5/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 433ms/step - acc: 0.5848 - loss: 0.2973 - learning_rate: 9.1201e-04
Epoch 6/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 433ms/step - acc: 0.6427 - loss: 0.2632 - learning_rate: 8.9125e-04
Epoch 7/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 434ms/step - acc: 0.6435 - loss: 0.2489 - learning_rate: 8.7096e-04
Epoch 8/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 433ms/step - acc: 0.6637 - loss: 0.2378 - learning_rate: 8.5114e-04
Epoch 9/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 68s 433ms/step - acc: 0.6649 - loss: 0.2304 - learning_rate

In [10]:
# Evaluating

pred = model.predict(x_test_2d)
prediction = np.zeros(np.shape(pred))
prediction[pred > 0.5] = 1
result(y_test, prediction)

36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 191ms/step
SNR
accuracy: 91.39 precision: 81.57 sensitivity: 93.08 specificity: 90.63 f1_score: 86.95
_______
AF
accuracy: 90.14 precision: 45.57 sensitivity: 34.62 specificity: 95.79 f1_score: 39.35
_______
IAVB
accuracy: 90.05 precision: 61.96 sensitivity: 42.54 specificity: 96.47 f1_score: 50.45
_______
LBBB
accuracy: 97.69 precision: 85.29 sensitivity: 58.0 specificity: 99.54 f1_score: 69.05
_______
RBBB
accuracy: 92.1 precision: 65.43 sensitivity: 46.49 specificity: 97.23 f1_score: 54.36
_______
SB
accuracy: 93.87 precision: 88.64 sensitivity: 86.43 specificity: 96.34 f1_score: 87.52
_______
STach
accuracy: 95.83 precision: 87.14 sensitivity: 92.92 specificity: 96.56 f1_score: 89.94
_______
accuracy: 70.69 %
AUC: 80.47 %
f1-weighted: 76.1 %
f1-micro: 77.52 %
f1-macro: 68.23 %
